In [1]:
from neo4j import GraphDatabase

In [3]:
url = "neo4j+ssc://1b36ae40.databases.neo4j.io"
username = "neo4j"
password = 'HSB9NaKfDmLFcGyXx_8MHyyL0Ap3mkd-1VIe1BV5XKs'

In [4]:
driver = GraphDatabase.driver(url,auth=(username, password))

In [5]:
def create_node(transaction,name,age):
    transaction.run("CREATE (:Person {name:$name,age:$age})", name=name,age=age)

with driver.session() as session:
    session.execute_write(create_node, "Karina",22)     #Creating nodes for users

In [6]:
with driver.session() as session:
    session.execute_write(create_node, "Maris",28)

In [7]:
with driver.session() as session:
    session.execute_write(create_node, "KKarlis",32)

In [8]:
with driver.session() as session:
    session.execute_write(create_node, "Sandra",20)

In [9]:
with driver.session() as session:
    session.execute_write(create_node, "Sanita",30)

In [10]:
with driver.session() as session:
    session.execute_write(create_node, "Davis",33)

In [11]:
def create_friendship(transaction,name1,name2):
    transaction.run("MATCH (person1:Person {name:$name1}), (person2:Person {name:$name2}) CREATE (person1)-[:FRIEND_OF]->(person2)", name1=name1,name2=name2)

with driver.session() as session:
    session.execute_write(create_friendship, "Davis","Sanita")  # creating relationship

In [12]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sanita","Davis")

In [13]:
with driver.session() as session:
    session.execute_write(create_friendship, "Davis","Kristine")

In [17]:
def delete_node(transaction, name):
    transaction.run("MATCH (person:Person {name:$name}) DETACH DELETE person", name=name)

with driver.session() as session:
    session.execute_write(delete_node,"Kristine") # deleting node and relationship

In [18]:
with driver.session() as session:
    session.execute_write(delete_node,"KKarlis")

In [19]:
with driver.session() as session:
    session.execute_write(create_node, "Karlis",30)

In [20]:
with driver.session() as session:
    session.execute_write(create_node, "Diana",29)

In [21]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sanita","Diana")

In [22]:
with driver.session() as session:
    session.execute_write(create_friendship, "Diana","Karlis")

In [23]:
with driver.session() as session:
    session.execute_write(create_friendship, "Karlis","Diana")

In [ ]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sanita","Davis")

In [26]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sandra","Maris")

In [25]:
with driver.session() as session:
    session.execute_write(create_friendship, "Maris","Karina")

In [27]:
with driver.session() as session:
    session.execute_write(create_friendship, "Maris","Karlis")

In [28]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sandra","Diana")

In [29]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sanita","Karina")

In [30]:
with driver.session() as session:
    session.execute_write(create_friendship, "Karina","Maris")

In [31]:
with driver.session() as session:
    session.execute_write(create_friendship, "Karina","Davis")

In [32]:
with driver.session() as session:
    session.execute_write(create_friendship, "Karina","Sandra")

In [33]:
with driver.session() as session:
    session.execute_write(create_friendship, "Sandra","Karlis")

Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation)

In [39]:
def get_person_with_friends(transaction, name):
    result = transaction.run("MATCH (person:Person {name:$name}) - [:FRIEND_OF] ->(friend:Person) RETURN person.name, person.age, friend.name, friend.age", name=name)
    return [{"name":record["person.name"], "age":record["person.age"], "friend_name":record["friend.name"], "friend_age":record["friend.age"]} for record in result]

In [42]:
with driver.session() as session:
    output = session.execute_write(get_person_with_friends, "Karina")
    print(output)

[{'name': 'Karina', 'age': 22, 'friend_name': 'Sandra', 'friend_age': 20}, {'name': 'Karina', 'age': 22, 'friend_name': 'Davis', 'friend_age': 33}, {'name': 'Karina', 'age': 22, 'friend_name': 'Maris', 'friend_age': 28}]


Retrieve common friends between two users in the graph

In [54]:
def common_friend(transaction, name1, name2):
    result = transaction.run("MATCH (person1:Person {name:$name1}), (person2:Person {name:$name2}) - [:FRIEND_OF] -> (friend:Person) RETURN person1.name, person2.name, friend.name", name1=name1, name2=name2)
    return [{"name1":record["person1.name"], "name2":record["person2.name"], "have_common_friend":record["friend.name"]} for record in result]

In [55]:
with driver.session() as session:
    output = session.execute_write(common_friend, "Karina", "Sanita")
    print(output)

[{'name1': 'Karina', 'name2': 'Sanita', 'have_common_friend': 'Karina'}, {'name1': 'Karina', 'name2': 'Sanita', 'have_common_friend': 'Diana'}, {'name1': 'Karina', 'name2': 'Sanita', 'have_common_friend': 'Davis'}]


Identifing the shortest path between two nodes

In [64]:
def find_shortest_path(transaction, name1, name2):
    result = transaction.run("MATCH (person1:Person {name:$name1}),(person2:Person {name:$name2}), path = SHORTESTPATH((person1) - [*] - (person2)) RETURN nodes(path), relationships(path)", name1=name1, name2=name2)
    return [{"nodes":record["nodes(path)"],"relationships":record["relationships(path)"]} for record in result]

In [68]:
with driver.session() as session:
    output = session.execute_write(find_shortest_path, "Karina", "Diana")
    nodes = output[0]["nodes"]
    relationships = output[0]["relationships"]

In [69]:
  for relationship in relationships:
    nodes = relationship.nodes
    print(nodes[0].get("name"),"->",nodes[1].get("name"))

Karina -> Sandra
Sandra -> Diana
